## Libraries

In [1]:
import random
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from dataclasses import dataclass
from typing import List

## Model

In [ ]:
RISK_LABELS = ["scam", "adult", "illegal", "spam", "low_info"]


class AuditorModel(nn.Module):
    def __init__(self, encoder_name: str, hidden_dim: int = 128):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        emb_dim = self.encoder.config.hidden_size

        self.shared = nn.Sequential(
            nn.Linear(emb_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
        )

        self.label_head = nn.Linear(hidden_dim, len(RISK_LABELS))
        self.score_head = nn.Linear(hidden_dim, 1)

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        out = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        pooled = out.last_hidden_state[:, 0]
        shared = self.shared(pooled)

        label_probs = torch.sigmoid(self.label_head(shared))
        risk_score = torch.sigmoid(self.score_head(shared)).squeeze(-1)

        return {
            "label_probs": label_probs,
            "risk_score": risk_score
        }

## Dataset

In [5]:
@dataclass
class AuditorSample:
    text: str
    risk_score: float       # [0, 1] noisy
    risk_labels: List[int]  # multi-hot
    source: str             # rule / synthetic / self_train


class AuditorDataset(Dataset):
    def __init__(self, samples: List[AuditorSample], tokenizer, max_len: int = 256):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        enc = self.tokenizer(
            s.text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        risk_labels = torch.tensor(
            [s.risk_labels[k] for k in RISK_LABELS],
            dtype=torch.float,
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "risk_score": torch.tensor(s.risk_score, dtype=torch.float),
            "risk_labels": risk_labels,
        }

## Train function

In [ ]:
def train_auditor(dataset, encoder_name: str):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AuditorModel(encoder_name).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    bce = torch.nn.BCELoss()
    mse = torch.nn.MSELoss()
    epochs = 10

    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        loop = tqdm.tqdm(loader, desc=f"Epoch {epoch + 1}")
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}

            out = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            loss_labels = bce(out["label_probs"], batch["risk_labels"])
            loss_score = mse(out["risk_score"], batch["risk_score"])

            loss = loss_labels + loss_score

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch + 1}: loss={epoch_loss / len(loader):.4f}")

    return model

## Generate synthetic data

In [7]:
SCAM_PHRASES = [
    "гарантированный доход",
    "переведите предоплату",
    "инвестиции без риска",
    "срочно, только сегодня"
]

LOW_INFO_TITLES = ["Продам", "Хорошее", "Отдам"]
SPAM_TOKENS = ["🔥🔥🔥", "!!!", "ТОЛЬКО СЕЙЧАС"]

NORMAL_TEMPLATES = [
    "Продаю {item}. Состояние {condition}. Использовался {time}. Причина продажи — {reason}."
]

ITEMS = ["велосипед", "телефон", "ноутбук"]
CONDITIONS = ["отличное", "хорошее"]
TIMES = ["1 год", "6 месяцев"]
REASONS = ["покупка нового", "не используется"]


def generate_ad(label: str) -> dict:
    if label == "scam":
        text = f"{random.choice(SCAM_PHRASES)} {random.choice(SCAM_PHRASES)}"
    elif label == "low_info":
        text = random.choice(LOW_INFO_TITLES)
    elif label == "spam":
        text = f"{random.choice(SPAM_TOKENS)} Купить {random.choice(SPAM_TOKENS)}"
    else:
        text = random.choice(NORMAL_TEMPLATES).format(
            item=random.choice(ITEMS),
            condition=random.choice(CONDITIONS),
            time=random.choice(TIMES),
            reason=random.choice(REASONS),
        )

    return {
        "text": text,
        "label": label
    }


def generate_dataset(n: int = 1000):
    labels = ["normal", "scam", "low_info", "spam"]
    return [generate_ad(random.choice(labels)) for _ in range(n)]


def weak_label(text: str):
    labels = {
        "scam": int("гарантир" in text or "предоплат" in text),
        "spam": int(text.count("!") > 2),
        "low_info": int(len(text) < 20),
        "adult": 0,
        "illegal": 0,
    }

    multi_hot = [labels[k] for k in RISK_LABELS]
    risk_score = float(max(multi_hot))

    return labels, risk_score

In [ ]:
ENCODER = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

tokenizer = AutoTokenizer.from_pretrained(ENCODER)

raws = generate_dataset(5000)

samples: List[AuditorSample] = []
for raw in raws:
    text = raw["text"]
    labels, score = weak_label(text)

    sample = AuditorSample(
        text=text,
        risk_score=score,
        risk_labels=labels,
        source="synthetic",
    )
    samples.append(sample)

dataset = AuditorDataset(samples, tokenizer)
model = train_auditor(dataset, ENCODER)

torch.save(model.state_dict(), "auditor.pt")

Epoch 0: 100%|██████████| 157/157 [01:44<00:00,  1.50it/s, loss=0.0919]


Epoch 0: loss=0.0000


Epoch 1: 100%|██████████| 157/157 [01:48<00:00,  1.45it/s, loss=0.0524]


Epoch 1: loss=0.0000


Epoch 2: 100%|██████████| 157/157 [01:47<00:00,  1.45it/s, loss=0.0258]


...

Epoch 7: loss=0.0000


Epoch 8: 100%|██████████| 157/157 [01:48<00:00,  1.45it/s, loss=0.00214]


Epoch 8: loss=0.0000


Epoch 9: 100%|██████████| 157/157 [01:48<00:00,  1.45it/s, loss=0.00238]


Epoch 9: loss=0.0000
